In [1]:
# !pip install chromedriver-binary==119.0.6045.105

In [1]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from urllib.parse import urlparse, parse_qs
import chromedriver_binary
import time
import pandas as pd

In [2]:
# race_info_df = pd.read_csv('.csv')
# race_info_df = race_info_df.drop('', axis=1)

In [3]:
# オプションを設定してヘッドレスモードを有効にする
chrome_options = Options()
chrome_options.add_argument("--headless")  # GUIをオフに
chrome_options.add_argument("--disable-gpu")  # GPUハードウェアアクセラレーションをオフに。Windows環境では必要な場合があります。
chrome_options.add_argument("--no-sandbox")  # サンドボックスプロセスを無効に。Linux環境の一部のセキュリティ制約を回避するために必要な場合があります。

# ヘッドレスモードでChromeを起動
driver = webdriver.Chrome(options=chrome_options)
# driver = webdriver.Chrome()

In [ ]:
# 'https://www.boatrace.jp/owpc/pc/race/odds3t?rno=12&jcd=01&hd=20131101'

**2017年3月10日以降にオッズデータが存在する**

In [12]:
data_list = []
for i in range(1, 13):
    url = 'https://www.boatrace.jp/owpc/pc/race/oddstf?rno=12&jcd=02&hd=20231107'
    target_url = f'https://www.boatrace.jp/owpc/pc/race/oddstf?rno={i}&jcd' + url.split('&jcd')[-1]
    
    # 日付
    date = url[-8:]
    
    # URLを解析してクエリパラメータを取得
    parsed_url = urlparse(url)
    query_params = parse_qs(parsed_url.query)

    # レース場ID
    place_id = query_params.get('jcd', [None])[0]
    
    # レース番号
    race_number = i

    # スクレイピング対象のURLにアクセス
    driver.get(target_url)

    table_x_path = '/html/body/main/div/div/div/div[2]/div[6]/div[1]/div[2]/table'
    table = driver.find_element(by=By.XPATH, value=table_x_path)
    tbody_elements = table.find_elements(by=By.TAG_NAME, value='tbody')

    for no in range(1, 7):
        d_list = []
        d_list.append(date)
        d_list.append(place_id)
        d_list.append(race_number)
        
        d_list.append(no) # 枠
        
        name = tbody_elements[no-1].find_element(by=By.TAG_NAME, value='span').text
        d_list.append(name)  # ボートレーサー
        
        odds = tbody_elements[no-1].find_elements(by=By.TAG_NAME, value='td')[2].text
        d_list.append(odds) # 単勝オッズ
        
        data_list.append(d_list)

In [13]:
columns = [
    'date', 'place_id', 'race_number', 'no', 'name', 'odds'
]

odds_df = pd.DataFrame(data_list, columns=columns)
odds_df.head()

,date,place_id,race_number,no,name,odds
0,20231107,02,1,1,品田 直樹,4.3
1,20231107,02,1,2,鹿島 敏弘,6.3
2,20231107,02,1,3,向後 龍一,1.9
3,20231107,02,1,4,橋本 英一,5.0
4,20231107,02,1,5,村岡 賢,12.3


In [14]:
odds_df

,date,place_id,race_number,no,name,odds
0,20231107,02,1,1,品田 直樹,4.3
1,20231107,02,1,2,鹿島 敏弘,6.3
2,20231107,02,1,3,向後 龍一,1.9
3,20231107,02,1,4,橋本 英一,5.0
4,20231107,02,1,5,村岡 賢,12.3
5,20231107,02,1,6,根岸 真優,6.3
6,20231107,02,2,1,前田 光昭,2.9
7,20231107,02,2,2,黒崎 竜也,2.4
8,20231107,02,2,3,河上 年昭,7.9
9,20231107,02,2,4,津久井 拓也,6.1
